# Load data

In [128]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

In [129]:
df = pd.read_csv(r'..\data\passing-grade.csv')
dummy = pd.read_csv(r"..\data\tryout_data.csv")

Dataset ini diambil dari dua sumber. Pertama, dataset utama (df) bersumber dari kaggle ("https://www.kaggle.com/datasets/rezkyyayang/passing-grade-utbk-in-science-major/data"). Kedua, dataset tambahan (dummy) yang didapatkan setelah proses ekstraksi data dari website (sc : "https://hasilto.bimbelssc.com/storage/ponorogo/intipa/data/TO_SNBT_JANUARI.html").

# EDA

In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   NO          500 non-null    int64  
 1   PTN         500 non-null    object 
 2   KODE PRODI  500 non-null    int64  
 3   NAMA PRODI  500 non-null    object 
 4   RATAAN      500 non-null    float64
 5   S.BAKU      500 non-null    float64
 6   MIN         500 non-null    float64
 7   MAX         500 non-null    float64
dtypes: float64(4), int64(2), object(2)
memory usage: 31.4+ KB


Dataset utama terdiri dari 500 baris (entries) dan 8 kolom, yaitu:
- NO: Tipe data int64, 500 data non-null.
- PTN: Tipe data object (string), 500 data non-null.
- KODE_PRODI: Tipe data int64, 500 data non-null.
- NAMA_PRODI: Tipe data object (string), 500 data non-null.
- RATAAN: Tipe data float64, 500 data non-null.
- SBAKU: Tipe data float64, 500 data non-null.
- MIN: Tipe data float64, 500 data non-null.
- MAX: Tipe data float64, 500 data non-null.

In [131]:
print(df['PTN'].unique())

['UNIVERSITAS INDONESIA' 'UNIVERSITAS AIRLANGGA' 'UNIVERSITAS PADJADJARAN'
 'UNIVERSITAS GADJAH MADA' 'UNIVERSITAS DIPONEGORO'
 'INSTITUT TEKNOLOGI BANDUNG' 'UNIVERSITAS BRAWIJAYA'
 'UNIVERSITAS SEBELAS MARET' 'UNIVERSITAS JENDERAL SOEDIRMAN'
 'UNIVERSITAS JEMBER' 'UNIVERSITAS UDAYANA'
 'INSTITUT TEKNOLOGI SEPULUH NOPEMBER' 'UNIVERSITAS SRIWIJAYA'
 'UPN "VETERAN" JAKARTA' 'UNIVERSITAS SUMATERA UTARA'
 'UNIVERSITAS ISLAM NEGERI MALANG' 'UNIVERSITAS ANDALAS'
 'UNIVERSITAS HASANUDDIN' 'UNIVERSITAS ISLAM NEGERI JAKARTA'
 'UNIVERSITAS MATARAM' 'INSTITUT PERTANIAN BOGOR' 'UNIVERSITAS LAMPUNG'
 'UNIVERSITAS SYIAH KUALA' 'UNIVERSITAS RIAU'
 'UNIVERSITAS PENDIDIKAN GANESHA' 'UNIVERSITAS MULAWARMAN'
 'UNIVERSITAS LAMBUNG MANGKURAT' 'UNIVERSITAS TANJUNGPURA'
 'UNIVERSITAS JAMBI' 'UNIVERSITAS SAM RATULANGI'
 'UNIVERSITAS NEGERI YOGYAKARTA' 'UNIVERSITAS NEGERI JAKARTA'
 'UNIVERSITAS CENDERAWASIH' 'UNIVERSITAS BENGKULU'
 'UNIVERSITAS MALIKUSSALEH' 'UNIVERSITAS PENDIDIKAN INDONESIA'
 'UNIVERSITAS NUS

pada dataset ini memiliki 50 perguruan tinggi yang daftarnya bisa dilihat pada output di atas.

In [132]:
rata2_per_PTN = df.groupby('PTN')['RATAAN'].mean()
urutan_PTN = rata2_per_PTN.sort_values(ascending=False)
df = df.set_index('PTN').loc[urutan_PTN.index].reset_index()

Mengelompokkan data berdasarkan kolom 'PTN' dan menghitung rata-rata nilai dari kolom 'RATAAN' untuk setiap PTN. Ini dilakukan untuk melihat PTN mana yang memiliki rata-rata nilai tertinggi dan terendah.

In [133]:
batasMean = df['RATAAN'].min()
batasSBaku = df['S.BAKU'].max()
batasMin = df['MIN'].min()
batasMax = df['MAX'].min()

print(f"nilai rataan terkecil : {batasMean}")
print(f"nilai S.baku terbesar : {batasSBaku}")
print(f"nilai MIN terkecil : {batasMin}")
print(f"nilai MAX terkecil : {batasMin}")

nilai rataan terkecil : 594.6
nilai S.baku terbesar : 29.58
nilai MIN terkecil : 581.92
nilai MAX terkecil : 581.92


Berikutnya dicari nilai minimum dari kolom "RATAAN", "MIN", dan "MAX". kemudian nilai maksimum dari kolom "S.BAKU" yang nantinya akan digunakan sebagai batasan 

# Prepration data dummy

Pada tahap ini kolom "participant_no", "name", "status_kelulusan", dan "timestamp" akan dihilangkan karena tidak dibutuhkan dalam klasifikasi

In [134]:
dummy = dummy.drop(columns=['participant_no', 'name', 'status_kelulusan', 'timestamp'])
dummy.head()

,pu,ppu,kmbm,pk,lit_ind,lit_ing,pm,total
0,"855,84","879,66","818,52","487,19","708,43","909,85","741,38","771,55"
1,"842,78","747,67","797,22","410,43","602,02","909,85","1000,00","758,57"
2,"787,79","545,34","783,33","593,05","775,49","742,14","1000,00","746,73"
3,"793,78","629,57","784,26","631,43","646,48","909,85","825,43","745,83"
4,"777,99","611,33","669,44","584,38","725,20","909,85","895,11","739,04"


Selanjutnya, akan dilakukan beberapa tahap preprocessing, yaitu:
1. mengganti nilai nol yang ditandai dengan "X" menjadi 0. 
2. kemudian menghapus spasi dan mengganti tanda koma menjadi titik. 
3. Selanjutnya mengubah tipe data menjadi numerik agar dapat dilakukan feature engineering.
4. mengisi missing value dengan nilai rata-rata karena metode ini mempertahankan distribusi data dan tidak mempengaruhi ukuran sampel secara signifikan. 
5. Selanjutnya menghapus data yang terduplikasi.
6. Akan dibuat kolom baru yang berisi nilai rata-rata, simpangan baku, total, terkecil dan terbesar pada setiap baris data untuk disesuaikan pada data utama.

In [135]:
# 1. Ganti 'X' dengan '0'
dummy = dummy.replace('X', 0)

# 2. Bersihkan data: hapus spasi & ganti koma dengan titik
dummy = dummy.applymap(lambda x: str(x).replace(' ', '').replace(',', '.'))

# 3. Konversi ke float per kolom (lebih aman)
for col in dummy.columns:
    dummy[col] = pd.to_numeric(dummy[col], errors='coerce')

# 4. Handle missing value
dummy = dummy.fillna(dummy.mean())

# 5. Handle duplicate
dummy = dummy.drop_duplicates()

# 6. Feature engineering
dummy['RATAAN'] = dummy[['pu', 'ppu', 'kmbm', 'pk', 'lit_ind', 'lit_ing', 'pm']].mean(axis=1)
dummy['S.BAKU'] = dummy[['pu', 'ppu', 'kmbm', 'pk', 'lit_ind', 'lit_ing', 'pm']].std(axis=1)
dummy['MIN'] = dummy[['pu', 'ppu', 'kmbm', 'pk', 'lit_ind', 'lit_ing', 'pm']].min(axis=1)
dummy['MAX'] = dummy[['pu', 'ppu', 'kmbm', 'pk', 'lit_ind', 'lit_ing', 'pm']].max(axis=1)

Pada tahap selanjutnya melibatkan penyesuaian nilai dalam beberapa kolom ('RATAAN', 'MIN', 'MAX', dan 'S.BAKU') berdasarkan ambang batas yang telah ditentukan, dengan tujuan menormalkan atau membersihkan data. Hal ini dilakukan untuk menghindari nilai yang terlalu rendah dibandingkan ambang batas, yang dianggap tidak realistis atau outlier. Dengan mengganti nilai tersebut dengan angka acak dalam rentang yang ditentukan, data menjadi lebih konsisten dan sesuai dengan ekspektasi analisis berikutnya. Rentang hingga 1000 dipilih sebagai batas atas yang wajar berdasarkan konteks data. Pada kolom 'S.BAKU' (standar deviasi) seharusnya tidak melebihi nilai tertentu yang telah ditentukan (29.58) untuk menjaga konsistensi variabilitas data. Nilai acak antara 0 dan batas dipilih untuk menormalisasi standar deviasi yang terlalu tinggi, yang bisa mengindikasikan noise atau kesalahan pengukuran. Selanjutnya, pembulatan dilakukan untuk menyederhanakan angka-angka dalam data, meningkatkan keterbacaan, dan mengurangi presisi yang tidak perlu.

In [136]:
import numpy as np

# Ambang batas
batas = {
    'RATAAN': batasMean,
    'MIN': batasMin,
    'MAX': batasMax,
    'S.BAKU':batasSBaku
}

# Ganti nilai yang kurang dari batas dengan nilai random dari (nilai min di kolom sampai 1000)
for kolom in ['RATAAN', 'MIN', 'MAX']:
    nilai_min = dummy[kolom].min()
    
    # Mask untuk nilai yang kurang dari batas
    mask = dummy[kolom] < batas[kolom]
    
    # Buat nilai random untuk posisi yang perlu diganti
    dummy.loc[mask, kolom] = np.random.uniform(batas[kolom], 1000, size=mask.sum())

nilai_min = dummy["S.BAKU"].min()
# Mask untuk nilai yang kurang dari batas
mask = dummy["S.BAKU"] > batas["S.BAKU"]    
# Buat nilai random untuk posisi yang perlu diganti
dummy.loc[mask, "S.BAKU"] = np.random.uniform(0, batas["S.BAKU"], size=mask.sum())

dummy[['RATAAN', 'S.BAKU', 'MIN', 'MAX']] = dummy[['RATAAN', 'S.BAKU','MIN', 'MAX']].round(2)



In [137]:
dummy.describe()

,pu,ppu,kmbm,pk,lit_ind,lit_ing,pm,total,RATAAN,S.BAKU,MIN,MAX
count,3247.00,3247.00,3247.00,3247.00,3247.00,3247.00,3247.00,3247.00,3247.00,3247.00,3247.00,3247.00
mean,312.99,328.84,413.59,189.29,448.14,466.89,265.86,346.52,794.08,14.86,791.26,793.56
std,171.59,155.31,180.92,106.56,206.72,274.91,182.81,133.64,118.52,8.65,120.14,104.04
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.77,594.66,0.01,582.02,621.44
25%,200.89,230.56,299.07,128.76,303.93,229.03,142.24,249.91,688.66,7.20,688.10,705.45
50%,312.59,336.93,426.85,185.71,440.95,474.84,233.48,348.90,793.68,14.95,793.12,789.31
75%,430.64,437.66,545.37,250.10,604.94,703.35,357.40,444.53,897.31,22.52,893.71,875.36
max,870.00,950.00,950.00,650.00,930.00,1000.00,1000.00,771.55,999.68,29.57,999.89,1000.00


# Modeling

Tahap ini melakukan modeling dengan random forest menggunakan data utama untuk memprediksi data dummy guna menciptakan data baru

In [138]:
X_train = df[['RATAAN', 'S.BAKU', 'MIN', 'MAX']]
y_train = df['PTN']
X_test = dummy[['RATAAN', 'S.BAKU', 'MIN', 'MAX']]

In [139]:
model = RandomForestClassifier(n_estimators=100, random_state=42)

model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [140]:
y_2 = model.predict(X_test)

dummy["PTN"] = y_2

membuat data baru yang berisi nantinya akan digabungkan dengan data utama

In [141]:
dummyBaru = dummy[["RATAAN", "S.BAKU", "MIN", "MAX", "PTN"]]
dummyBaru

,RATAAN,S.BAKU,MIN,MAX,PTN
0,771.55,9.11,587.93,909.85,UNIVERSITAS BRAWIJAYA
1,758.57,2.22,928.56,1000.00,UNIVERSITAS AIRLANGGA
2,746.73,18.60,857.64,1000.00,UNIVERSITAS PADJADJARAN
3,745.83,14.93,629.57,909.85,UNIVERSITAS INDONESIA
4,739.04,17.96,584.38,909.85,INSTITUT TEKNOLOGI SEPULUH NOPEMBER
...,...,...,...,...,...
3244,964.94,12.14,984.48,937.68,UNIVERSITAS AIRLANGGA
3245,942.20,12.12,715.04,857.92,UNIVERSITAS AIRLANGGA
3246,888.17,10.17,706.78,753.43,UNIVERSITAS JENDERAL SOEDIRMAN
3247,683.12,8.64,862.05,627.12,UNIVERSITAS JENDERAL SOEDIRMAN


menggabungkan data baru dan data utama

In [142]:
gabungan = pd.concat([df[["RATAAN", "S.BAKU", "MIN", "MAX", "PTN"]], dummyBaru], ignore_index=True)
gabungan

,RATAAN,S.BAKU,MIN,MAX,PTN
0,747.93,19.63,724.38,798.55,UNIVERSITAS INDONESIA
1,712.41,24.13,685.96,798.66,UNIVERSITAS INDONESIA
2,716.32,29.35,681.72,782.33,UNIVERSITAS INDONESIA
3,681.69,13.46,663.43,716.90,UNIVERSITAS INDONESIA
4,690.30,19.63,663.30,734.80,UNIVERSITAS INDONESIA
...,...,...,...,...,...
3742,964.94,12.14,984.48,937.68,UNIVERSITAS AIRLANGGA
3743,942.20,12.12,715.04,857.92,UNIVERSITAS AIRLANGGA
3744,888.17,10.17,706.78,753.43,UNIVERSITAS JENDERAL SOEDIRMAN
3745,683.12,8.64,862.05,627.12,UNIVERSITAS JENDERAL SOEDIRMAN


melakukan preparation pada data baru, yaitu membagi data menjadi 80% data latih dan 20% data uji

In [143]:
X = gabungan.drop(columns='PTN')  
y = gabungan['PTN']               

# Split: 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

melakukan modeling dengan random forest

In [144]:
model = RandomForestClassifier(n_estimators=100, random_state=42)

model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [145]:
y_pred = model.predict(X_test)

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"\nAccuracy Score: {accuracy*100:.0f}%")
print(f"F1 Score: {f1*100:.0f}")

Confusion Matrix:
[[ 2  0  3 ...  1  0  0]
 [ 0 13  2 ...  0  0  0]
 [ 0  2 13 ...  0  0  0]
 ...
 [ 1  0  0 ...  1  0  0]
 [ 2  0  0 ...  0  1  0]
 [ 1  0  0 ...  0  0  0]]

Classification Report:
                                             precision    recall  f1-score   support

                   INSTITUT PERTANIAN BOGOR       0.17      0.14      0.15        14
                 INSTITUT TEKNOLOGI BANDUNG       0.48      0.43      0.46        30
        INSTITUT TEKNOLOGI SEPULUH NOPEMBER       0.42      0.50      0.46        26
                      UNIVERSITAS AIRLANGGA       0.94      0.92      0.93       109
                        UNIVERSITAS ANDALAS       0.00      0.00      0.00         3
                      UNIVERSITAS BRAWIJAYA       0.78      0.74      0.76        70
                   UNIVERSITAS CENDERAWASIH       0.00      0.00      0.00         1
                     UNIVERSITAS DIPONEGORO       0.53      0.50      0.52        16
                    UNIVERSITAS GADJ

akurasi pada model adalah 75% angka ini mungkin cukup rendah, tetapi dapat ditoleransi karena beberapa perguruan tinggi memiliki kesamaan pola sehingga sulit untuk membedakan secara pasti berdasarkan fitur yang digunakan. Dengan akurasi 75%, model masih dapat dianggap berguna untuk analisis awal atau sebagai panduan, terutama jika data tambahan atau fitur yang lebih spesifik dapat diterapkan untuk meningkatkan pemisahan antar kelas di masa depan.

In [146]:
import joblib

joblib.dump(model, 'model_klasifikasi.pkl')

['model_klasifikasi.pkl']